# Generic ML Script 2

In [1]:
from propscore import PropensityScore
import random
import pandas as pd
import sklearn
import sklearn.model_selection
import numpy as np
import statistics as stats
import statsmodels.api as sm
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from causalinference import CausalModel
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import datasets, ensemble
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet

Ignore the bottom example code.

# Main function

To run Generic ML single, we need the dataset to include the propensity scores and fixed effects (if any). 

Propensity scores 
1. Calculated from the package propscore or otherwise given by experiment design (e.g. if 50% of individuals are randomly treated, propensity score is 0.5)
2. Column for propensity score should be labelled as "propscore"

Fixed effects
1. Dummy variables for all states except for the reference state (to avoid multicollinearity)

In [2]:
def Generic_ML_single(df, treatment, outcome, controls, iterations = 10, model = "random_forest", alpha = 0.05, k = 5, fixed_effects = None): 
    '''
    Runs the whole generic ML algorithm for a ML model and returns a list of datasets for all parameters.  
    '''
    
    data_HET_loop = []
    data_ATE_loop = []
    lambda1_loop = []

    data_GATES_loop = []
    lambda2_loop = []

    data_CLAN_loop = []


    for x in range(iterations): 
        main, aux = sklearn.model_selection.train_test_split(df, train_size = 0.5, random_state = x)
        main2 = ML_estimator(main, aux, model, treatment, outcome, controls, fixed_effects) 
    
        # BLP
        res_BLP, lambda1 = BLP(main2, treatment, outcome, alpha)
        data_HET, data_ATE = BLP_to_storage(res_BLP)
        data_HET_loop.append(data_HET)
        data_ATE_loop.append(data_ATE)
        lambda1_loop.append(lambda1)
    
        #GATES
        res_GATES, t_test_GATES, lambda2 = GATES(main2, k, treatment, outcome, alpha) 
        data_GATES = GATES_to_storage(res_GATES, t_test_GATES, alpha)
        data_GATES_loop.append(data_GATES)
        lambda2_loop.append(lambda2)
    
        # CLAN
        data_CLAN = CLAN(main2, treatment, controls, alpha)
        data_CLAN_loop.append(data_CLAN)

        # BLP
        data_HET_array = np.array(data_HET_loop)
        data_HET_final = np.median(data_HET_array, axis = 0)
        data_HET_final[2] = np.minimum(1, data_HET_final[2] *2)

        data_ATE_array = np.array(data_ATE_loop)
        data_ATE_final = np.median(data_ATE_array, axis = 0)
        data_ATE_final[2] = np.minimum(1, data_ATE_final[2] * 2)    
    
    
    df_BLP = data_BLP_to_df(data_HET_loop, data_ATE_loop)
    df_GATES = data_GATES_to_df(data_GATES_loop, k)
    df_CLAN = data_CLAN_to_df(data_CLAN_loop, controls = controls)
    
    lambda1 = np.mean(lambda1_loop)
    lamda2 = np.mean(lambda2_loop)
    
    df_GATES = update_GATES_df(df_GATES, alpha).round(3)
    df_CLAN = update_CLAN_df(df_CLAN, controls, alpha).round(3)
    
    summary = [df_BLP, df_GATES, df_CLAN, lambda1, lambda2]
    return summary

# BLP 

In [3]:
def BLP(df, treatment, outcome, alpha): 
    
    '''
    Finds the Best Linear Predictor (BLP) of the Average Treatment Effect(ATE).
    
    Returns dataframe of summary results, whose parameters can be used to obtain BLP of Conditional ATE(CATE).
    Contains: 
        Estimator Coefficients of Term 2 and 3 
        Standard Error 
        p values 
        Confidence Interval (lower and upper bounds)
    
    Returns lambda1 - value to help choose the best ML method 
    
    Parameters 
    ----------
    df -- (main) dataframe which must contain the following items: 
        propensity score 
        B - proxy predictor for BCA 
        S - proxy predictor for CATE
        treatment 
    outcome 
    alpha -- significance level 
        
    '''
    
    term2 = df[treatment] - df['propscore']
    S = df['S']
    term3 = term2 * (S - np.mean(S))
    
    
    combined = df.copy()
    combined.loc[:,'term2'] = term2 
    combined.loc[:,'term3'] = term3
    combined.loc[:,'ones'] = 1 
    
    X_reg = combined[['B', 'S', 'ones', 'term2', 'term3']]
    y = combined[outcome]
    
    regBLP = sm.OLS(y, X_reg)
    res_BLP = regBLP.fit()
    
    res_BLP = results_summary_to_dataframe(res_BLP, alpha)
    
    lambda1 = res_BLP.iloc[-1,0] * stats.variance(S)   
    return res_BLP, lambda1
    
def results_summary_to_dataframe(results, alpha):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    std_err = results.bse
    
    crit_val = norm.ppf(1-alpha/2) 

    lb = coeff - std_err * crit_val
    ub = coeff + std_err * crit_val
    

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "lb":lb,
                               "ub":ub,
                               "std_err":std_err, 
                                })

    #Reordering...
    results_df = results_df[["coeff","std_err","pvals","lb","ub"]]
    return results_df

def BLP_to_storage(res_BLP):
    
    '''
    Takes the output of BLP and store them as lists
        res_BLP - summary table containing parameters to construct BLP, along with their p-values, standard errors and lower and upper bounds
        
    Returns 2 lists data_HET and data_ATE whose array-equivalent is of dimension (1 variable, 5 attributes)
    '''
    
    # HET parameter 
    HET = res_BLP.iloc[-1,0]
    HET_se = res_BLP.iloc[-1,1]
    HET_pvals = res_BLP.iloc[-1, 2]
    HET_lb = res_BLP.iloc[-1, 3]
    HET_ub = res_BLP.iloc[-1, 4]

    # ATE 
    ATE = res_BLP.iloc[-2,0]
    ATE_se = res_BLP.iloc[-2,1]
    ATE_pvals = res_BLP.iloc[-2,2]
    ATE_lb = res_BLP.iloc[-2,3]
    ATE_ub = res_BLP.iloc[-2,4]
    
    # Storage
    
    data_HET = [HET, HET_se, HET_pvals, HET_lb, HET_ub]
    data_ATE = [ATE, ATE_se, ATE_pvals, ATE_lb, ATE_ub]

    return data_HET, data_ATE

# GATES

In [4]:
def GATES(df, k, treatment, outcome,  alpha): 
    '''
    Returns summary statistics, whose results can give us the average treatment effect 
    for most and least affected group, and the difference between them. 
    
    Contains: 
        Estimator Coefficients  
        Standard Error 
        p values 
        Confidence Interval (lower and upper bounds)
        
    Returns lambda2 - value to help choose the best ML method 
    
    Parameters 
    ----------
    df -- (main) dataframe which must contain the following items: 
        propensity score 
        B - proxy predictor for BCA 
        S - proxy predictor for CATE
        treatment 
        
    k -- number of groups 
    treatment
    outcome 
    alpha -- significance level 
    '''
    
    combined = df.copy()
    term2 = df[treatment] - df['propscore']
    combined.loc[:,'term2'] = term2
    combined.loc[:,'ones'] = 1
    
    groups = groups_multiply(df, group_create(k, df), treatment, k)
    combined = pd.concat([combined,groups], axis = 1) 
  
    controls_GATES = ["B", "S", "ones"] + ["G" + str(i) for i in range(1,k+1)]
    X_GATES = combined[controls_GATES] # modify for auto selection of columns
    y = combined[outcome]
    
    regGATES = sm.OLS(y, X_GATES)
    res_GATES = regGATES.fit()
    
    # Hypothesis testing 
    hypothesis = "(G1 = " + "G" + str(k) + ")" # G1 = G{k}
    t_test_html = res_GATES.t_test(hypothesis).summary().as_html()
    t_test = pd.read_html(t_test_html, header=0, index_col=0)[0]
    
    res_GATES = results_summary_to_dataframe(res_GATES, alpha)
    
    lambda2 = res_GATES.iloc[3:, 0].mean()**2 / k
    
    return res_GATES, t_test, lambda2
    

def group_create(k, df): 
    '''
    Returns quantiles of the variable 'S', encoded into dummy variables
    '''
    breaks = df['S'].quantile(np.linspace(0,1,(k+1)))
    breaks.iloc[0,] = breaks.iloc[0,] - 0.001 
    breaks.iloc[k,] = breaks.iloc[k,] - 0.001 
    
    combined = df.copy()
    combined['Groups'] = pd.cut(x= df['S'], bins = breaks) # this will fail if there are too many groups
    groups = pd.get_dummies(combined['Groups'])
    
    return groups

def groups_multiply(df, groups, treatment, k):
    '''
    Multiply dataframe generated by function group_create with term 2 and rename columns 
    '''
    
    combined = df.copy()
    term2 = df[treatment] - df['propscore']
    combined.loc[:,'term2'] = term2
    
    groups = np.multiply(groups, combined['term2'].values.reshape(len(df.index),1))
    groups.columns = ["G" + str(i) for i in range(1,k+1)] 
    
    return groups

def GATES_to_storage(res_GATES, t_test_GATES, alpha):
    
    '''
    Takes the output of GATES and store them as lists, whereby the output refers to: 
        res_GATES - summary table containing parameters to construct GATES, along with their p-values and standard errors 
        t_test_GATEs - t test table to determine if G1 = Gk 
    
    Returns a list whose array-equivalent is dimension of (# of variables, # of attributes )
    '''
    
    # Most affected group 
    gamma1 = res_GATES.iloc[3,0]
    gamma1_se = res_GATES.iloc[3,1]
    gamma1_pvals = res_GATES.iloc[3,2]
    gamma1_lb = res_GATES.iloc[3,3]
    gamma1_ub = res_GATES.iloc[3,4]

    # Least affected group 
    gammak = res_GATES.iloc[-1,0]
    gammak_se = res_GATES.iloc[-1,1]
    gammak_pvals = res_GATES.iloc[-1,2]
    gammak_lb = res_GATES.iloc[-1,3]
    gammak_ub = res_GATES.iloc[-1,4]
    
    # Difference between most and least affected group 
  
    crit_val = norm.ppf(1-alpha/2) 

    gamma_diff = abs(t_test_GATES.iloc[0,0])
    gamma_diff_se = t_test_GATES.iloc[0,1]
    gamma_diff_pvals = t_test_GATES.iloc[0,3] 
    gamma_diff_lb = gamma_diff - crit_val * gamma_diff_se
    gamma_diff_ub = gamma_diff + crit_val * gamma_diff_se
    
    data_gamma1 = [gamma1, gamma1_se, gamma1_pvals, gamma1_lb, gamma1_ub]
    data_gammak = [gammak, gammak_se, gammak_pvals, gammak_lb, gammak_ub]
    data_gamma_diff = [gamma_diff, gamma_diff_se, gamma_diff_pvals, gamma_diff_lb, gamma_diff_ub]

    data_gamma = [data_gamma1, data_gammak, data_gamma_diff]
    
    return data_gamma

# CLAN

In [5]:
def CLAN(df, treatment, controls, alpha, k = 5):
    
    '''
    Finds the average of the controls for the most and least affected groups and the difference between them.  
    
    Returns this as a list. 
    
    Parameters 
    ----------
    df -- (main) dataframe which must contain the following items: 
        propensity score 
        B - proxy predictor for BCA 
        S - proxy predictor for CATE
        treatment 
        controls 
    
    treatment (string) 
    controls (list of column names)
    alpha -- significance level 
    k -- number of groups
    '''
    
    data_CLAN_loop = []
    for x in controls: 
        res_CLAN, t_test = CLAN_single(df, treatment, x, alpha, k)
        data_CLAN = CLAN_to_storage(res_CLAN, t_test, alpha)
        data_CLAN_loop.append(data_CLAN)
    return data_CLAN_loop

def CLAN_single(df, treatment, control, alpha, k):
    
    '''
    Gives the average characteristic for a control for the most and least affected group and the difference between these groups. 
        Uses group membership as defined in GATES and runs a regression of a control on dummy variables for group membership. 
        The coefficients are the effect of group membership on the control variable or the mean of the control for that particular group. 
        Difference between most and least affected is derived from a t test. 
    
    Returns 2 dataframes. 
        The first dataframe is for the most and least affected group. 
        The second dataframe is the difference between them. 
    
    Parameters 
    ----------
    df -- (main) dataframe which must contain the following items: 
        propensity score 
        B - proxy predictor for BCA 
        S - proxy predictor for CATE
        treatment 
        control 
        
    treatment
    control 
    k -- number of groups
    '''
    
    combined = df.copy()
    term2 = combined[treatment] - combined['propscore']
    combined.loc[:,'term2'] = term2
    combined.loc[:,'ones'] = 1

    groups_df = group_create(k, combined)
    groups_df.columns = ["G" + str(i) for i in range(1,k+1)]
    combined = pd.concat([combined , groups_df], axis = 1) 

    X_control = combined[["G" + str(i) for i in range(1,k+1)]] 
    y_control = combined[[control]]

    reg_CLAN = sm.OLS(y_control, X_control)
    res_CLAN = reg_CLAN.fit()
    
    hypothesis = "(G1 = " + "G" + str(k) + ")" # G1 = G{k}
    t_test_html = res_CLAN.t_test(hypothesis).summary().as_html()
    t_test = pd.read_html(t_test_html, header=0, index_col=0)[0]

    res_CLAN = results_summary_to_dataframe(res_CLAN, alpha)
    
    return res_CLAN, t_test

def CLAN_to_storage(res_CLAN, t_test, alpha):
    
    '''
    Takes the summary results of CLAN_single and store them as lists. 
    
    Parameters 
    ----------
    res_CLAN -- first dataframe returned from CLAN_single 
    t_test -- second dataframe returned from CLAN_single
    alpha -- significance level 
    '''
    
    h_coeff = res_CLAN.iloc[0,0]
    h_se = res_CLAN.iloc[0,1]
    h_pvals = res_CLAN.iloc[0,2]
    h_lb = res_CLAN.iloc[0,3]
    h_ub = res_CLAN.iloc[0,4]
    data_h = [h_coeff, h_se, h_pvals, h_lb, h_ub]
    
    l_coeff = res_CLAN.iloc[-1,0]
    l_se = res_CLAN.iloc[-1,1]
    l_pvals = res_CLAN.iloc[-1,2]
    l_lb = res_CLAN.iloc[-1,3]
    l_ub = res_CLAN.iloc[-1,4]
    data_l = [l_coeff, l_se, l_pvals, l_lb, l_ub]
    
    crit_val = norm.ppf(1-alpha/2) 
    
    diff_coeff = t_test.iloc[0,0]
    diff_se = t_test.iloc[0,1]
    diff_pvals = t_test.iloc[0,3]
    diff_lb = diff_coeff - crit_val * diff_se
    diff_ub = diff_coeff + crit_val * diff_se
    data_diff = [diff_coeff, diff_se, diff_pvals, diff_lb, diff_ub]
    
    data_CLAN = data_h, data_l, data_diff
    return data_CLAN

# Converting data to dataframes and Misc Functions

In [6]:
def data_BLP_to_df(data_HET_loop, data_ATE_loop): 
    '''
    Takes the data of BLP stored as a list, find its median over different iterations and adjusts p values. 
    
    Returns it as a dataframe 
    '''
    
    data_HET_array = np.array(data_HET_loop)
    data_HET_final = np.median(data_HET_array, axis = 0)
    data_HET_final[2] = np.minimum(1, data_HET_final[2] *2)

    data_ATE_array = np.array(data_ATE_loop)
    data_ATE_final = np.median(data_ATE_array, axis = 0)
    data_ATE_final[2] = np.minimum(1, data_ATE_final[2] * 2)   
    
    df_ATE = pd.DataFrame(data_ATE_final, 
                     index = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                     columns = ['ATE'])

    df_HET = pd.DataFrame(data_HET_final, 
                     index = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                     columns = ['HET'])

    frames = [df_ATE, df_HET]
    
    df_BLP = pd.concat(frames, axis = 1)
    
    return df_BLP
    
def data_GATES_to_df(data_GATES_loop, groups): 
    '''
    Takes the data of GATES stored as a list, find its median over different iterations and adjusts p values. 
    
    Returns it as a dataframe 
    '''
    
    # GATES 
    data_GATES_array = np.array(data_GATES_loop)
    data_GATES_final = np.median(data_GATES_array, axis = 0)
    data_GATES_final[:, 2] = np.minimum(1, data_GATES_final[:, 2]* 2)
    
    df_GATES = pd.DataFrame(data_GATES_final, 
                        columns = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                        index = ['least affected(' + str(100/groups) + "%)", 'most affected(' + str(100 - 100/groups) + "%)", 'most - least affected'])
    
    return df_GATES.transpose()

def data_CLAN_to_df(data_CLAN_loop, controls): 
    '''
    Takes the data of GATES stored as a list, find its median over different iterations and adjusts p values. 
    
    Returns it as a dataframe 
    '''
    
    # CLAN 
    data_CLAN_array = np.array(data_CLAN_loop) 


    data_CLAN_final = np.median(data_CLAN_array, axis = 0) # This code is technically wrong as we take the upper medians for the lower bounds
    data_CLAN_final[0,2,:] = np.minimum(1, data_CLAN_final[0,2,:] * 2)
    
    list = []
    for x in controls: 
        list1 = ['Least affected (' + str(x) + ")", 'Most affected (' + str(x) + ")", 'Most - Least affected (' + str(x) + ")" ]
        list.append(list1)
    
    flattened_list = [y for x in list for y in x]

    data_CLAN_new = data_CLAN_final.reshape(-1,5)
    df_CLAN = pd.DataFrame(data_CLAN_new, 
                      columns = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                      index = flattened_list)

    return df_CLAN

def update_GATES_df(df, alpha):
    
    '''
    Take the GATES dataframe and ensures that the difference between the most and least affected group 
    is the ATE of most affected - ATE of least affected
    '''
    
    crit_val = norm.ppf(1-alpha/2) 
    
    df.iloc[0,-1] = df.iloc[0,1] - df.iloc[0,0]
    df.iloc[3,-1] = df.iloc[0,-1] -  df.iloc[1,-1] *  crit_val
    df.iloc[4,-1] = df.iloc[0,-1] +  df.iloc[1,-1] *  crit_val
    
    return df

def update_CLAN_df(df, controls, alpha):
    
    '''
    Take the CLAN dataframe and ensures that the difference between the most and least affected group 
    is the ATE of most affected - ATE of least affected
    '''
    
    crit_val = norm.ppf(1-alpha/2) 
    # Most - least 
    
    for i, x in enumerate(controls, start = 1): 
        y = 3*(i)
        df.iloc[(y-1) , 0] = df.iloc[(y -2),0] - df.iloc[(y -3),0]
        df.iloc[(y-1),3] = df.iloc[(y -1),0] -  df.iloc[(y -1),1] *  crit_val
        df.iloc[(y-1),4] = df.iloc[(y -1),0] +  df.iloc[(y -1),1] *  crit_val
    
    return df

def create_states(df, fixed_effects):
    '''
    Create binary variables for the fixed effects and drop one of them
    '''
    states = pd.get_dummies(df[fixed_effects])
    states.drop(states.columns[0], axis = 1, inplace = True)
    
    return states 

# ML estimators

In [7]:
def ML_estimator(main, aux, model, treatment, outcome, controls, fixed_effects = None):
    '''
    Returns the main dataset combined with B and S, which are proxy predictors for BCA and CATE respectively 
    
    Parameters 
    ----------
    main: main dataset which must contain treatment and outcome
    aux: auxilliary dataset which must contain treatment and outcome
    model - in string format 
        supported models = ["random_forest", "SVM", "gradient_boost", "neural_net", "ElasticNet"]
    treatment - in str
    outcome - in str 
    com
    
    # need to set the seed of the ML_estimators
    
    '''
    
    # Initialization
    if fixed_effects == None: 
        aux0 = aux[aux[treatment] == 0]
        aux1 = aux[aux[treatment] == 1]
        X_aux0 = aux0.loc[:,[treatment] + controls]
        y_aux0 =aux0[outcome]
        X_aux1 = aux1.loc[:,[treatment] + controls]
        y_aux1 =aux1[outcome]
    
        X_main = main.loc[:,[treatment] + controls]
        y_main = main[outcome]
        
    else: 
        states = create_states(main, fixed_effects) # need to have same column names as in the original dataframe
        
        cols = [treatment] + controls + list(states.columns)
        aux0 = aux[aux[treatment] == 0]
        aux1 = aux[aux[treatment] == 1]
        X_aux0 = aux0.loc[:,cols]
        y_aux0 =aux0[outcome]
        X_aux1 = aux1.loc[:,cols]
        y_aux1 =aux1[outcome]
    
        X_main = main.loc[:,cols]
        y_main = main[outcome]
    
    
    # Model 
    if model == "random_forest": 
        combined = random_forest(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "SVM": 
        combined = SVM(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "gradient_boost": 
        combined = gradient_boost(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "neural_net": 
        combined = neural_net(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "ElasticNet": 
        combined = ElasticNet(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    
    # Add variance
    if stats.variance(combined['S']) == 0 : 
        combined['S'] = combined['S'] + np.random.normal(0,0.1, len(combined['S'])) 
    if stats.variance(combined['B']) == 0 : 
        combined['B'] = combined['B'] + np.random.normal(0,0.1, len(combined['B'])) 
        
    return combined

def random_forest(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):
    
    # Model 
    clf = RandomForestRegressor(max_depth=2, random_state=0)
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def SVM(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):

    # Model 
    clf = svm.SVR()
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def gradient_boost(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):

    
    params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}
    
    # Model 
    clf = ensemble.GradientBoostingRegressor(**params)
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def neural_net(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):
    
    # Model 
    clf = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def ElasticNet(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):
        
    # Model 
    clf = sklearn.linear_model.ElasticNet()
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined